In [23]:
# %load features100.py
import csv
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

from sklearn.metrics import accuracy_score,auc, precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import joblib

import csv
import pandas as pd
import numpy as np
from sklearn import model_selection
from tpot import TPOTClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier
#from pyearth import Earth
from sklearn.cross_decomposition import PLSRegression


from sklearn.feature_selection import SelectKBest

from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from skfeature.function.statistical_based import t_score
from skfeature.function.statistical_based import gini_index
from skfeature.function.similarity_based import fisher_score
from skfeature.function.similarity_based import reliefF

from sklearn.feature_selection import mutual_info_classif
from skfeature.function.information_theoretical_based import LCSI
from skfeature.function.information_theoretical_based import MIM
from skfeature.function.information_theoretical_based import MIFS
from skfeature.function.information_theoretical_based import MRMR
from skfeature.function.information_theoretical_based import CIFE
from skfeature.function.information_theoretical_based import JMI
from skfeature.function.information_theoretical_based import CMIM
from skfeature.function.information_theoretical_based import ICAP
from skfeature.function.information_theoretical_based import DISR

from scipy.stats import wilcoxon

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

import warnings


file_train = open("./csv/train.csv")
csv_f = csv.reader(file_train)
features = next(csv_f)
dataset = pd.read_csv("./csv/train.csv", names=features, usecols=range(1,6132), dtype=np.float64, skiprows=1, low_memory=False)
# INITIALIZING, CLEANING, AND STRATIFYING DATASET
dataset["outcome"] = pd.to_numeric(dataset["outcome"], errors='coerce')
dataset.dropna(axis=1, thresh=2, inplace=True)
dataset.dropna(how='all',thresh = 20,inplace=True)
train_feature = np.array(dataset)
wh_inf = np.isinf(train_feature)
train_feature[wh_inf]=0
wh_nan = np.isnan(train_feature)
train_feature[wh_nan]=0

file_validate = open("./csv/validation.csv")
csv_f = csv.reader(file_validate)
features = next(csv_f)
dataset = pd.read_csv("./csv/validation.csv", names=features, usecols=range(1,6132), dtype=np.float64, skiprows=1, low_memory=False)
# INITIALIZING, CLEANING, AND STRATIFYING DATASET
dataset["outcome"] = pd.to_numeric(dataset["outcome"], errors='coerce')
dataset.dropna(axis=1, thresh=2, inplace=True)
validate_feature = np.array(dataset)
wh_inf = np.isinf(validate_feature)
validate_feature[wh_inf]=0
wh_nan = np.isnan(validate_feature)
validate_feature[wh_nan]=0


file_test = open("./csv/test.csv")
csv_f = csv.reader(file_test)
features = next(csv_f)
dataset = pd.read_csv("./csv/test.csv", names=features, usecols=range(1,6132), dtype=np.float64, skiprows=1, low_memory=False)
# INITIALIZING, CLEANING, AND STRATIFYING DATASET
dataset["outcome"] = pd.to_numeric(dataset["outcome"], errors='coerce')
dataset.dropna(axis=1, thresh=2, inplace=True)
test_feature = np.array(dataset)
wh_inf = np.isinf(test_feature)
test_feature[wh_inf]=0
wh_nan = np.isnan(test_feature)
test_feature[wh_nan]=0


#only use image features
X_train = train_feature[:,:6130]
Y_train = train_feature[:,6130]
Y_train = Y_train.astype('int32')

X_validate = validate_feature[:,:6130]
Y_validate = validate_feature[:,6130]
Y_validate = Y_validate.astype('int32')

X_test = test_feature[:,:6130]
Y_test = test_feature[:,6130]
Y_test = Y_test.astype('int32')
seed = 7

np.random.seed(seed)
np.random.shuffle(X_train) 
np.random.seed(seed)
np.random.shuffle(Y_train)


pipe = joblib.load('handpkl/NNetFSCR100.pkl')

Y_pred = pipe.predict(X_validate)
Y_prob = pipe.predict_proba(X_validate)
print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
print("AUC"+repr(roc_auc_score(Y_validate,Y_prob[:,1])))
print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
print("Specificity" + repr(tn/(tn+fp)))

        
Y_pred = pipe.predict(X_test)
Y_prob = pipe.predict_proba(X_test)
print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
print("AUC: " + repr(roc_auc_score(Y_test, Y_prob[:,1])))
print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
print("Specificity" + repr(tn/(tn+fp)))
fpr, tpr, thresholds= precision_recall_curve(Y_test, Y_prob[:,1], pos_label=1)
auc = auc(tpr,fpr)
print(auc)


Accuracy: 0.7745098039215687
Average Precision Score: 0.6588732744683498
Kappa: 0.5390962671905697
Hamming Loss: 0.22549019607843138
AUC0.862460815047022
Sensitivity0.7272727272727273
Specificity0.8103448275862069
Accuracy: 0.74
Average Precision Score: 0.6288995215311004
Kappa: 0.46457990115321257
Hamming Loss: 0.26
AUC: 0.7727272727272727
Sensitivity0.6363636363636364
Specificity0.8214285714285714
0.7538330981670445


In [12]:
print(Y_test)

[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0]


In [25]:
import sys
#p is proportion of trials that were successes
#n is the number of trials
import math
def adjusted_wald(p, n, z=1.96):
    p_adj = (n * p + (z**2)/2)/(n+z**2)
    n_adj = n + z**2
    span = z * math.sqrt(p_adj*(1-p_adj)/n_adj)
    return max(0, p_adj - span), min(p_adj + span, 1.0)
print("({:.2f}-{:.2f})".format(*adjusted_wald(float(0.74), float(50))))
print("({:.2f}-{:.2f})".format(*adjusted_wald(float(0.64), float(22))))
print("({:.2f}-{:.2f})".format(*adjusted_wald(float(0.82), float(28))))

(0.60-0.84)
(0.43-0.81)
(0.64-0.92)


Accuracy: 0.7941176470588235
Average Precision Score: 0.6818775995246583
Kappa: 0.5814771395076201
Hamming Loss: 0.20588235294117646
AUC0.8499216300940439
Sensitivity0.7727272727272727
Specificity0.8103448275862069
Accuracy: 0.72
Average Precision Score: 0.6054545454545455
Kappa: 0.42622950819672123
Hamming Loss: 0.28
AUC: 0.7987012987012988
Sensitivity0.6363636363636364
Specificity0.7857142857142857
0.7697348763359573


In [33]:
tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
print(tn,fp,fn,tp)

23 5 8 14


In [37]:
# 50,22,28
# 0.74,0.64,0.82
# 0.72,0.64,0.79
import sys
from scipy.stats import binom_test
#x accuracy
#n = number of trials
#p accuracy compare to
x = 0.82
n =28
p = 0.79

print(binom_test(int(x * n), n, p))

1.0


In [40]:
#0.875,1.0,1.0
print(len(X_train[0]))

6130


In [38]:
best = SelectKBest(fisher_score.fisher_score, k=100)
best.fit(X_train, Y_train)
idx = best.get_support(indices=True)

In [47]:
for i in idx:
    if i<3064:
        i = i + 23
    else:
        i = i - 3064
        if i>23:
            i = i + 23
            
    print(i)

179
488
665
899
1046
1163
1211
1226
1227
1244
1245
1295
1397
1498
1506
1698
1705
1731
1745
1807
1827
1850
1886
1927
1958
2227
2342
2374
2388
2415
2531
2591
2661
2668
2716
2912
3010
3077
117
134
158
222
238
264
281
282
288
309
310
370
371
374
441
480
510
524
541
676
777
778
858
880
968
1048
1056
1160
1330
1354
1417
1418
1526
1530
1560
1878
1879
1880
1945
1951
2042
2080
2084
2098
2119
2131
2151
2157
2187
2196
2204
2284
2425
2668
2693
2698
2766
2802
2891
2956
2963
2999


In [ ]:
179
488
665
899
1046
1163
1211
1226
1227
1244
1245
1295
1397
1498
1506
1698
1705
1731
1745
1807
1827
1850
1886
1927
1958
2227
2342
2374
2388
2415
2531
2591
2661
2668
2716
2912
3010
3077
3204
3221
3245
3309
3325
3351
3368
3369
3375
3396
3397
3457
3458
3461
3528
3567
3597
3611
3628
3763
3864
3865
3945
3967
4055
4135
4143
4247
4417
4441
4504
4505
4613
4617
4647
4965
4966
4967
5032
5038
5129
5167
5171
5185
5206
5218
5238
5244
5274
5283
5291
5371
5512
5755
5780
5785
5853
5889
5978
6043
6050
6086

In [1]:
import csv

In [17]:
file = open('./csv/experts_outcome.csv','r')
expert = []
for index,row in enumerate(csv.reader(file)):
    if index == 0:
        continue
    tmp = [int(row[1]),int(row[2]),int(row[3]),int(row[4]),int(row[5])]
    expert.append(tmp)

[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 1, 0, 0, 1]
[1, 1, 0, 0, 1]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 1, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 1, 0, 0]
[0, 1, 1, 1, 1]
[1, 0, 0, 1, 1]
[1, 1, 1, 1, 1]
[0, 1, 1, 0, 1]
[1, 1, 1, 1, 1]
[1, 0, 1, 1, 1]
[1, 1, 1, 1, 1]
[0, 1, 1, 1, 1]
[1, 1, 0, 1, 1]
[0, 1, 1, 1, 1]
[1, 1, 1, 0, 1]
[1, 0, 1, 1, 1]
[1, 0, 0, 0, 1]
[0, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 0, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 0]
[1, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 0, 0]


In [25]:
import numpy as np
file = open('./csv/experts_outcome.csv','r')
expert = []
for index,row in enumerate(csv.reader(file)):
    if index == 0:
        continue
    tmp = [int(row[1]),int(row[2]),int(row[3]),int(row[4]),int(row[5])]
    expert.append(tmp)
expert = np.array(expert)
Y_test = expert[:,4]
Y_pred = expert[:,3]
print(Y_test)
print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
print("AUC: " + repr(roc_auc_score(Y_test, Y_prob[:,1])))
print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
print(tn,fp,fn,tp)
print("Specificity" + repr(tn/(tn+fp)))
fpr, tpr, thresholds= precision_recall_curve(Y_test, Y_prob[:,1], pos_label=1)
auc_ = auc(tpr,fpr)
print(auc_)


[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0]
Accuracy: 0.84
Average Precision Score: 0.7568181818181817
Kappa: 0.6721311475409836
Hamming Loss: 0.16
AUC: 0.7727272727272727
Sensitivity0.7727272727272727
25 3 5 17
Specificity0.8928571428571429


TypeError: 'numpy.float64' object is not callable

In [ ]:
Accuracy:    0.76(0.62-0.86)  0.84(0.71-0.92)  0.84(0.71-0.92)  0.84(0.71-0.92)  avg: 0.82(0.69-0.90)
Sensitivity: 0.73(0.52-0.87)  0.82(0.61-0.93)  0.73(0.52-0.87)  0.77(0.56-0.90)       0.76(0.55-0.89)
Specificity: 0.79(0.61-0.90)  0.86(0.68-0.95)  0.93(0.76-0.99)  0.89(0.72-0.97)       0.87(0.69-0.96)

In [30]:
import sys
#p is proportion of trials that were successes
#n is the number of trials
import math
def adjusted_wald(p, n, z=1.96):
    p_adj = (n * p + (z**2)/2)/(n+z**2)
    n_adj = n + z**2
    span = z * math.sqrt(p_adj*(1-p_adj)/n_adj)
    return max(0, p_adj - span), min(p_adj + span, 1.0)
print("({:.2f}-{:.2f})".format(*adjusted_wald(float(0.82), float(50))))
print("({:.2f}-{:.2f})".format(*adjusted_wald(float(0.76), float(22))))
print("({:.2f}-{:.2f})".format(*adjusted_wald(float(0.87), float(28))))

(0.69-0.90)
(0.55-0.89)
(0.69-0.96)


In [43]:
# 50,22,28
# 0.74,0.64,0.82
# 0.72,0.64,0.79
import sys
from scipy.stats import binom_test
#x accuracy
#n = number of trials
#p accuracy compare to
x = 0.82
n =28
p = 0.87

print(binom_test(int(x * n), n, p))

0.2526064810983525


In [ ]:
hand 0.14,0.21,0.25
tpot 0.09,0.21,0.25